In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


The above code is for logging the process in the console.

# Gensim corpora and dictionary

How to tokenize words and convert them to vectors along with their occurance in the document using doc2bow function for the Dictionary in corpora of gensim and how to use token2id to make 2d vectors.

In [1]:
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
               "A survey of user opinion of computer system response time",
               "The EPS user interface management system",
               "System and human system engineering testing of EPS",
               "Relation of user perceived response time to error measurement",
               "The generation of random binary unordered trees",
               "The intersection graph of paths in trees",
               "Graph minors IV Widths of trees and well quasi ordering",
               "Graph minors A survey"]

stoplist = set('for a fo the and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]
#print(texts)

from collections import defaultdict

frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]for text in texts]
#collecting values which are repetitive more than once.

from pprint import pprint
#pprint(texts)

dictionary = corpora.Dictionary(texts)
#To save the dictionary to dict file for further use of this document.
#uncomment the below to save the dictionary to the drive.

##dictionary.save('kathitest.dict')
print(dictionary.token2id)

new_doc = "Human computer interface human system interface"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'of': 4, 'user': 5, 'system': 6, 'response': 7, 'time': 8, 'eps': 9, 'trees': 10, 'graph': 11, 'minors': 12}
[(0, 2), (1, 2), (2, 1), (6, 1)]


[(0, 2), (1, 2), (2, 1), (6, 1)]
In the above list the first part of the tuple is the id from the 2d vector and the second part is the occurance.

In [5]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary.token2id, end="\n")
for c in corpus:
    print(c, end="\n")
for text in texts:
    print(text, end="\n")

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'of': 4, 'user': 5, 'system': 6, 'response': 7, 'time': 8, 'eps': 9, 'trees': 10, 'graph': 11, 'minors': 12}
[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)]
[(1, 1), (5, 1), (6, 1), (9, 1)]
[(0, 1), (4, 1), (6, 2), (9, 1)]
[(4, 1), (5, 1), (7, 1), (8, 1)]
[(4, 1), (10, 1)]
[(4, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1), (12, 1)]
[(3, 1), (11, 1), (12, 1)]
['human', 'interface', 'computer']
['survey', 'of', 'user', 'of', 'computer', 'system', 'response', 'time']
['eps', 'user', 'interface', 'system']
['system', 'human', 'system', 'of', 'eps']
['of', 'user', 'response', 'time']
['of', 'trees']
['graph', 'of', 'trees']
['graph', 'minors', 'of', 'trees']
['graph', 'minors', 'survey']


# Handling corpus without consuming large RAM.

In [31]:
class MyCorpus(object):
    
    def __iter__(self):
        for line in open("mycorpus.txt"):
            yield dictionary.doc2bow(line.lower().split())
            
            
corpus_memory_friendly = MyCorpus()
"""This will return generator instead of the entire list,
which we can iterate effectively with out getting to memorize the elements which are read."""

print(corpus_memory_friendly)

In [32]:
#save the corpus to the drive.
corpora.MmCorpus.serialize("mycorpus.mm", corpus_memory_friendly)

for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


# Getting bag of words from the corpus text

In [30]:

from six import iteritems


stoplist = set('for a of the and to in'.split())

dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))

stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]

print(stop_ids)

once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

print(once_ids)

dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print(dictionary)
#saving in to a dictionary file.
dictionary.save("mycorpus.dict")

[10, 16, 3, 35, 19, 24, 8]
[1, 3, 4, 5, 7, 12, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 37, 38, 39, 40, 41]
Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [10]:
print(dictionary.token2id)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'of': 4, 'user': 5, 'system': 6, 'response': 7, 'time': 8, 'eps': 9, 'trees': 10, 'graph': 11, 'minors': 12}


In [12]:
ka_new_document = "Human computer interface."
ka_new_vector = dictionary.doc2bow(ka_new_document.lower().split())
print(ka_new_vector)

[(0, 1), (2, 1)]


The '.' full stop is not considered in converting to vectors.
with out '.', the generated vector will be as.

In [13]:
ka_new_document = "Human computer interface"
ka_new_vector = dictionary.doc2bow(ka_new_document.lower().split())
print(ka_new_vector)

[(0, 1), (1, 1), (2, 1)]


In [17]:
import gensim
import numpy as np
np.random.seed(1)
numpy_matrix = np.random.randint(10, size=[5,2])  # random matrix as an example
print("The random numpy_matrix is: ",numpy_matrix)
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)

The random numpy_matrix is:  [[5 8]
 [9 5]
 [0 0]
 [1 7]
 [6 9]]


NameError: name 'number_of_corpus_features' is not defined

In [19]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)  # random sparse matrix as example
print("The sparse matrix is :",scipy_sparse_matrix)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
print("The corpus is: ", corpus)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)
print(scipy_csc_matrix)



The sparse matrix is : 
The corpus is:  <gensim.matutils.Sparse2Corpus object at 0x00000022703E9320>



# corpus conversion using numpy and scipy.sparse random generators

In [33]:
ka_dict = corpora.Dictionary.load("mycorpus.dict")
ka_corpus = corpora.MmCorpus("mycorpus.mm")
print(ka_dict.token2id, end="\n")
for ka_corpora in ka_corpus:
    print(ka_corpora)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}
[(0, 1.0), (1, 1.0), (2, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)]
[(1, 1.0), (4, 1.0), (5, 1.0), (8, 1.0)]
[(0, 1.0), (5, 2.0), (8, 1.0)]
[(4, 1.0), (6, 1.0), (7, 1.0)]
[(9, 1.0)]
[(9, 1.0), (10, 1.0)]
[(9, 1.0), (10, 1.0), (11, 1.0)]
[(3, 1.0), (10, 1.0), (11, 1.0)]


# The above is the way to load the corpus and the dictionary ids from the drive.

The corpus loaded in the 'ka_corpus' is the generator, which then looped to get the individual vectors. The occurance is printed in float values.
In the below the code the generator is retured.

In [34]:
print(ka_corpus)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


# The models in gensim has TfidfModel function.
tfidf is treated as a read-only object that can be used to convert any vector from the old representation (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):

In [36]:
from gensim import models
tfidf = models.TfidfModel(ka_corpus)

In [38]:
test_vec = [(0,1),(1,1)]
print(tfidf[test_vec])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [39]:
for ka_corpora in ka_corpus:
    print(tfidf[ka_corpora])

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(2, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.3244870206138555), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.44424552527467476)]
[(1, 0.5710059809418182), (4, 0.4170757362022777), (5, 0.4170757362022777), (8, 0.5710059809418182)]
[(0, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(4, 0.45889394536615247), (6, 0.6282580468670046), (7, 0.6282580468670046)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(3, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


The transformation of vectors to the realistic values could be done either using the above code or using the below by applying to the whole corpus.

In [40]:
transformed_vectors = tfidf[ka_corpus]
for vector in transformed_vectors:
    print(vector)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(2, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.3244870206138555), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.44424552527467476)]
[(1, 0.5710059809418182), (4, 0.4170757362022777), (5, 0.4170757362022777), (8, 0.5710059809418182)]
[(0, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(4, 0.45889394536615247), (6, 0.6282580468670046), (7, 0.6282580468670046)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(3, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]
